In [19]:
from hidden import IBM_tone_analyzer_api
import pandas as pd

# IBM Watson Tone Analyzer

To read these notes directly from IBM, please visit this website: https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/python.html?python#error-handling. Otherwise, most of the notes seen bellow have been taken directly from the website above and placed here to aid the reader.

The IBM Watson Tone Analyzer service uses linguistic analysis to detect emotional and language tones in written text. The service can analyze tone at both the **document** and **sentence levels**.

## Authentication

IBM Cloud is migrating to token-based Identity and Access Management (IAM) authentication. With some service instances, you authenticate to the API by using IAM. You can pass either a bearer token in an Authorization header or an API key. If you pass in the API key, the SDK manages the lifecycle of the tokens.

In [8]:
from watson_developer_cloud import ToneAnalyzerV3

In [11]:
tone_analyzer = ToneAnalyzerV3(
    version = '2017-09-21',
    iam_apikey = IBM_tone_analyzer_api,
    url = 'https://gateway-wdc.watsonplatform.net/tone-analyzer/api'
)

## Service endpoint

The service endpoint is based on the location of the service instance. If your API endpoint URL differs from the default, you must set your endpoint. 

To find out which URL to use, view the service credentials by clicking the service instance on the Dashboard. Set the correct service ```URL``` by using the url parameter when you create the service instance or by calling the ```set_url()``` method of the service instance.

In [12]:
url = 'https://gateway-wdc.watsonplatform.net/tone-analyzer/api'

## Versioning

API requests require a version parameter that takes a date in the format ```version=YYYY-MM-DD```. When IBM changes the API in a **backwards-incompatible** way, they release a new version date.

Specify the version to use on API requests with the version parameter when you create the service instance. The service uses the API version for the date you specify, or the most recent version before that date. Don't default to the current date. Instead, specify a date that matches a version that is compatible with your app, and don't change it until your app is ready for a later version.

In [13]:
version = '2017-09-21'

## Data handling

#### Data Collection
By default, all Watson services log requests and their results. **Logging is done only to improve the services for future users. The logged data is not shared or made public.**

To prevent IBM from accessing your data for general service improvements, set the ```X-Watson-Learning-Opt-Out``` header parameter to ```true``` when you create the service instance. (Any value other than false or 0 disables request logging.) You can set the header using the ```set_default_headers``` method of the service object.

In [14]:
tone_analyzer.set_default_headers({'x-watson-learning-opt-out': "true"})

## Methods

#### Analyze general tone

Use the general purpose endpoint to analyze the tone of your input content. **The service analyzes the content for emotional and language tones**. 
- The method always analyzes the tone of the **full document; by default**, it also analyzes the **tone of each individual sentence of the content**.

You can submit no more than 128 KB of total input content and no more than 1000 individual sentences in JSON, plain text, or HTML format. The service analyzes the first 1000 sentences for document-level analysis and only the first 100 sentences for sentence-level analysis.

Per the JSON specification, the default character encoding for JSON content is effectively always UTF-8; per the HTTP specification, the default encoding for plain text and HTML is ISO-8859-1 (effectively, the ASCII character set). When specifying a content type of plain text or HTML, include the charset parameter to indicate the character encoding of the input text; for example: Content-Type: text/plain;charset=utf-8. For text/html, the service removes HTML tags and analyzes only the textual content.

#### Request

tone(self, tone_input, content_type, sentences=None, tones=None, content_language=None, accept_language=None, **kwargs)

**ToneInput**:

| Name        | Description     | 
| ------------- |:-------------:| 
| text (str):     | The input content that the service is to analyze. |



In [30]:
import json
from watson_developer_cloud import ToneAnalyzerV3

text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'

tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
j =(json.dumps(tone_analysis, indent=2))

# Function to Distill the Overarching Document Tones Into a Pandas DF

In [16]:
def text_analysis_to_pd(tone_analysis):
    return pd.DataFrame.from_dict(tone_analysis['document_tone']['tones'])

#### example

In [20]:
text_analysis_to_pd(tone_analysis)

,score,tone_id,tone_name
0,0.616500,sadness,Sadness
1,0.829888,analytical,Analytical


# Functions to Distill the Sentence Tones Into a Pandas DF

In [23]:
def get_score(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Score {idx}: {score['score']} "
    return output

In [24]:
def get_tone(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Tone {idx}: {score['tone_name']} "
    return output

In [205]:
def tone_to_doc_analysis(tone_analysis):
    df = text_analysis_to_pd(tone_analysis)
    return df[['tone_name','score']]

def text_to_sentence_analysis(text):
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
    df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])
    df['score'] = df['tones'].apply(get_score)
    df['tone'] = df['tones'].apply(get_tone)
    return df[['sentence_id', 'text', 'score', 'tone']]

In [21]:
df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])

In [26]:
df['score'] = df['tones'].apply(get_score)
df['tone'] = df['tones'].apply(get_tone)

# Sample Pipeline: MBA student's "failure" essay 

#### Student's Essay

In [98]:
t = """ In my 2nd year in university, my 2 study partners and I were all working for software companies. We frequently discussed ways to make quantum career leaps. One that fascinated us was starting our own company.

One day we came up with an idea that would increase sales for consumer goods retailers and simultaneously decrease monthly consumer expenses. Each day, we polished our idea together for a couple hours.

After 2 weeks, I decided to get outside feedback. I looked for people who had at least 10 years experience in consumer goods. Finally, I convinced a friend, to connect me with a board member of the 2nd largest consumer goods retailer in my country.

I presented our business model to the board member, and he instructed his right-hand to set us meetings with managers who could evaluate our plans. Over the next month, we went to one meeting after another. The responses varied from enthusiasm to skepticism. Each time, we improved our presentation according to the feedback.

Finally, I managed to set a meeting with the previous CEO of the largest consumer goods retailer. He concluded our meeting with: “Guys, in my opinion, it’s not going to work”.

I couldn’t say if it was the pressure from school and work or the CEO’s negative feedback, but since that meeting, I wasn’t able to motivate the team to go on. We consciously gave up. 2 years later, one of my teammates called out of the blue: “check out this link…it works!”. I think he expected me to feel disappointment. Actually, I felt pride – my first business attempt was viable after all."""

#### Overarching Document Tones

In [207]:
tone_to_doc_analysis(tone_analysis)

,tone_name,score
0,Joy,0.605044
1,Sadness,0.507810
2,Analytical,0.858529


#### Sentence Tones

In [99]:
text_to_sentence_analysis(t)

,sentence_id,text,score,tone
0,0,"In my 2nd year in university, my 2 study partn...",Score 1: 0.559715 Score 2: 0.660207,Tone 1: Sadness Tone 2: Confident
1,1,We frequently discussed ways to make quantum c...,,
2,2,One that fascinated us was starting our own co...,,
3,3,One day we came up with an idea that would inc...,Score 1: 0.541591,Tone 1: Analytical
4,4,"Each day, we polished our idea together for a ...",Score 1: 0.684799,Tone 1: Joy
5,5,"After 2 weeks, I decided to get outside feedback.",Score 1: 0.901894 Score 2: 0.874372,Tone 1: Analytical Tone 2: Confident
6,6,I looked for people who had at least 10 years ...,Score 1: 0.508012,Tone 1: Joy
7,7,"Finally, I convinced a friend, to connect me w...",Score 1: 0.772407 Score 2: 0.579367,Tone 1: Joy Tone 2: Analytical
8,8,I presented our business model to the board me...,Score 1: 0.503542,Tone 1: Analytical
9,9,"Over the next month, we went to one meeting af...",,


In [227]:
tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()

In [101]:
tone_analysis['sentences_tone'][0]['tones']

[{'score': 0.559715, 'tone_id': 'sadness', 'tone_name': 'Sadness'},
 {'score': 0.660207, 'tone_id': 'confident', 'tone_name': 'Confident'}]

In [102]:
def find_sentence_tone(tone_analysis, tone):
    for d in tone_analysis['sentences_tone']:
        if len(d['tones']) > 0:
            for sent in d['tones']:
                if sent['tone_name'] == tone:
                    print(sent['score'])
                else:
                    print(f'This sentence was not {tone}')
        else:
            print(f'This sentence was not {tone}')

In [155]:
def find_sentence_tone(tone_analysis, tone):
    for d in tone_analysis['sentences_tone']:
        if len(d['tones']) > 0:
            for sent in d['tones']:
                if sent['tone_name'] == tone:
                    sentence = d['text']
                    score = sent['score']
                    level = label_score(sent['score'])
                    color = add_color(label_score(sent['score']), tone)
                    yield {
                        'sentence' : sentence,
                        'score' : score,
                        'level' : level,
                        'color' : color
                           }
                else:
                    sentence = d['text']
                    level = 'Zero'
                    yield {
                        'sentence' : sentence,
                        'score' : 0,
                        'level' : 'Zero',
                        'color' : add_color('Zero', tone)
                           }
        else:
            sentence = d['text']
            yield {
                   'sentence' : sentence,
                   'score' : 0,
                   'level' : 'Zero',
                   'color' : add_color('Zero', tone)
                    }

In [187]:
def label_score(score):
    if score < .5:
        return 'Zero'
    elif .5 <= score <= .75:
        return 'Mid'
    else:
        return 'High'

def add_color(level, tone):
    if tone == 'Analytical':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#afd5fd'
        else:
            return '#6eaff1'
    if tone == 'Joy':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#fffd9d'
        else:
            return '#f7f338'
    if tone == 'Sadness':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#eaebec'
        else:
            return '#cacaca'
    if tone == 'Anger':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#f3c9c9'
            else:
                return '#e28d8d'
    if tone == 'Tentative':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#e5cdf1'
            else:
                return '#cc97e6'
    if tone == 'Confident':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#bcf5c9'
            else:
                return '#44e268'
    if tone == 'Fear':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#f3dee7'
            else:
                return '#f1a6c6'

In [191]:
analytical_df = pd.DataFrame(list(find_sentence_tone(tone_analysis, 'Analytical')))

In [198]:
import pickle
pickle.dump(tone_analysis, open("tone_analysis.p", "wb")) 

In [199]:
favorite_color = pickle.load(open("tone_analysis.p", "rb"))

In [225]:
analytical_df[analytical_df['level'] == 'High'].values[0][0]

'#6eaff1'

In [195]:
text_to_doc_analysis(t).to_pickle("doc.pkl")

In [209]:
tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()

In [226]:
text = """I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.

Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity.

But one hundred years later, the Negro still is not free; one hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination; one hundred years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity; one hundred years later, the Negro is still languished in the corners of American society and finds himself in exile in his own land.

So we’ve come here today to dramatize a shameful condition. In a sense we’ve come to our nation’s capital to cash a check. When the architects of our republic wrote the magnificent words of the Constitution and the Declaration of Independence, they were signing a promissory note to which every American was to fall heir. This note was the promise that all men, yes, black men as well as white men, would be guaranteed the unalienable rights of life, liberty, and the pursuit of happiness.

It is obvious today that America has defaulted on this promissory note in so far as her citizens of color are concerned. Instead of honoring this sacred obligation, America has given the Negro people a bad check, a check which has come back marked “insufficient funds.” But we refuse to believe that the bank of justice is bankrupt. We refuse to believe that there are insufficient funds in the great vaults of opportunity of this nation. And so we have come to cash this check, a check that will give us upon demand the riches of freedom and the security of justice.

We have also come to this hallowed spot to remind America of the fierce urgency of now. This is no time to engage in the luxury of cooling off or to take the tranquilizing drug of gradualism. Now is the time to make real the promises of democracy; now is the time to rise from the dark and desolate valley of segregation to the sunlit path of racial justice; now is the time to lift our nation from the quicksands of racial injustice to the solid rock of brotherhood; now is the time to make justice a reality for all of God’s children. It would be fatal for the nation to overlook the urgency of the moment. This sweltering summer of the Negro’s legitimate discontent will not pass until there is an invigorating autumn of freedom and equality. Nineteen sixty-three is not an end, but a beginning. And those who hope that the Negro needed to blow off steam and will now be content, will have a rude awakening if the nation returns to business as usual. There will be neither rest nor tranquility in America until the Negro is granted his citizenship rights. The whirlwinds of revolt will continue to shake the foundations of our nation until the bright day of justice emerges.

But there is something that I must say to my people, who stand on the worn threshold which leads into the palace of justice. In the process of gaining our rightful place, we must not be guilty of wrongful deeds. Let us not seek to satisfy our thirst for freedom by drinking from the cup of bitterness and hatred. We must forever conduct our struggle on the high plane of dignity and discipline. We must not allow our creative protests to degenerate into physical violence. Again and again we must rise to the majestic heights of meeting physical force with soul force. The marvelous new militancy, which has engulfed the Negro community, must not lead us to a distrust of all white people. For many of our white brothers, as evidenced by their presence here today, have come to realize that their destiny is tied up with our destiny. And they have come to realize that their freedom is inextricably bound to our freedom. We cannot walk alone. And as we walk, we must make the pledge that we shall always march ahead. We cannot turn back.

There are those who are asking the devotees of Civil Rights, “When will you be satisfied?” We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality; we can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities; we cannot be satisfied as long as the Negro’s basic mobility is from a smaller ghetto to a larger one; we can never be satisfied as long as our children are stripped of their selfhood and robbed of their dignity by signs stating “For Whites Only”; we cannot be satisfied as long as the Negro in Mississippi cannot vote, and the Negro in New York believes he has nothing for which to vote. No! no, we are not satisfied, and we will not be satisfied until “justice rolls down like waters and righteousness like a mighty stream.”

I am not unmindful that some of you have come here out of great trials and tribulations.  Some of you have come fresh from narrow jail cells. Some of you have come from areas where your quest for freedom left you battered by the storms of persecution and staggered by the winds of police brutality. You have been the veterans of creative suffering. Continue to work with the faith that unearned suffering is redemptive. Go back to Mississippi. Go back to Alabama. Go back to South Carolina. Go back to Georgia. Go back to Louisiana. Go back to the slums and ghettos of our Northern cities, knowing that somehow this situation can and will be changed.  Let us not wallow in the valley of despair.

I say to you today, my friends, so even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream deeply rooted in the American dream. I have a dream that one day this nation will rise up and live out the true meaning of its creed, “We hold these truths to be self-evident, that all men are created equal.” I have a dream that one day on the red hills of Georgia, sons of former slaves and the sons of former slaveowners will be able to sit down together at the table of brotherhood. I have a dream that one day even the state of Mississippi, a state sweltering with the heat of injustice, sweltering with the heat of oppression, will be transformed into an oasis of freedom and justice. I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.

I HAVE A DREAM TODAY!

I have a dream that one day down in Alabama — with its vicious racists, with its Governor having his lips dripping with the words of interposition and nullification — one day right there in Alabama, little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.

I HAVE A DREAM TODAY!

I have a dream that one day every valley shall be exalted, and every hill and mountain shall be made low. The rough places will be plain and the crooked places will be made straight, “and the glory of the Lord shall be revealed, and all flesh shall see it together.”

This is our hope. This is the faith that I go back to the South with. With this faith we will be able to hew out of the mountain of despair a stone of hope.  With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brother-hood. With this faith we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day.  And this will be the day. This will be the day when all of God’s children will be able to sing with new meaning, “My country ’tis of thee, sweet land of liberty, of thee I sing. Land where my father died, land of the pilgrim’s pride, from every mountainside, let freedom ring.” And if America is to be a great nation, this must become true.

So let freedom ring from the prodigious hilltops of New Hampshire; let freedom ring from the mighty mountains of New York; let freedom ring from the heightening Alleghenies of Pennsylvania; let freedom ring from the snow-capped Rockies of Colorado; let freedom ring from the curvaceous slopes of California. But not only that. Let freedom ring from Stone Mountain of Georgia; let freedom ring from Lookout Mountain of Tennessee; let freedom ring from every hill and mole hill of Mississippi. “From every mountainside, let freedom ring.”

And when this happens, and when we allow freedom to ring, when we let it ring from every village and every hamlet, from every state and every city, we will be able to speed up that day when all of God’s children, black men and white men, Jews and Gentiles, Protestants and Catholics, will be able to join hands and sing in the words of the old Negro spiritual: “Free at last. Free at last. Thank God Almighty, we are free at last.”"""

In [228]:
tone_analysis

{'document_tone': {'tones': [{'score': 0.517444,
    'tone_id': 'sadness',
    'tone_name': 'Sadness'},
   {'score': 0.635536, 'tone_id': 'joy', 'tone_name': 'Joy'},
   {'score': 0.629539, 'tone_id': 'confident', 'tone_name': 'Confident'}]},
 'sentences_tone': [{'sentence_id': 0,
   'text': 'I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.',
   'tones': [{'score': 0.887421, 'tone_id': 'joy', 'tone_name': 'Joy'},
    {'score': 0.574477, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]},
  {'sentence_id': 1,
   'text': 'Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation.',
   'tones': [{'score': 0.588023, 'tone_id': 'joy', 'tone_name': 'Joy'}]},
  {'sentence_id': 2,
   'text': 'This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice.',
   'tone

In [245]:
def find_doc_tones(tone_analysis):
    tones = []
    for d in tone_analysis['sentences_tone']:
        if len(d['tones']) > 0:
            for sent in d['tones']:
                if sent['tone_name'] not in tones:
                    tone = sent['tone_name']
                    tones.append(tone)
                    score = sent['score']
                    level = label_score(sent['score'])
                    color = add_color(label_score(sent['score']), tone)
                    yield {
                        'tone_name' : tone
                           }
                else:
                    continue
        else:
            continue

In [246]:
def doc_tone_finder(tone_analysis):
    return pd.DataFrame(list(find_doc_tones(tone_analysis)))

In [247]:
doc_tone_finder(tone_analysis)

,tone_name
0,Joy
1,Analytical
2,Sadness
3,Confident
4,Tentative
5,Anger
